In [36]:
import matplotlib.pyplot as plt
import matplotlib
from pathlib2 import Path
import numpy as np
import pandas as pd

In [37]:
figures_dir = Path('')

In [175]:

font = {
        'size'   : 12,
        }

#matplotlib.rc('font', **font)
#matplotlib.rcParams["text.usetex"] = True

matplotlib.use("pgf")

matplotlib.rcParams.update({
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.texsystem': 'lualatex',
    'pgf.preamble': r'\usepackage{fontspec} \setmainfont{Times New Roman} \usepackage{amsmath}',
})
matplotlib.rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
matplotlib.rc('text.latex',preamble=r'\usepackage[russian]{babel} \usepackage{amsmath}')

def read_data():
    df_binomial = pd.read_csv('p_binomial.dat', header=None, names=['Y', 'p'], index_col=False, sep=' ')
    df_y_init = pd.read_csv('p_y_init_x_1.dat', header=None, names=['Y', 'p'], index_col=False, sep=' ')
    df_reactions = pd.read_csv('ssa_4_reactions_pdf_tau_50_al_10_1_Y0_20.csv',sep=';')
    df_reactions = pd.read_csv('ssa_4_reactions_pdf_tau_50_al_10_1_Y0_20_.csv',sep=';')
    df_markov = pd.read_csv('markov.dat', header=None, names=['t', 'p'], index_col=False, sep=' ')
    df_reactions = pd.read_csv('ssa_4_reactions_pdf_tau_50_al_10_0_Y0_20.csv',sep=';')
    df_reactions = pd.read_csv('ssa_4_reactions_pdf_tau_50_al_10_0_Y0_20_CY_Inf.csv',sep=';')
    df_reactions_data = pd.read_csv('ssa_4_reactions_pdf_tau_50_al_10_0_Y0_20.csv_ccw.dat', header=None, names=['t', 'p'], index_col=False, sep='\t')
    
    return df_binomial, df_y_init, df_reactions, df_markov, df_reactions_data


def make_plot():
    fig = plt.figure(figsize=(6, 4), dpi=300)
    ax = plt.gca()
    df_binomial, df_y_init, df_reactions, df_markov, df_reactions_data = read_data()
    mnorm = (df_markov['t'] * df_markov['p']).sum()
    #plt.plot(df_y_init['Y'], df_y_init['p'], color='#1F78B4')
    
    t = np.linspace(1, 100, 2)
    y = t ** -2.18# * 100
    plt.plot(t, y, '--', color='#6A3D9A', lw=2, label='$t^{-\gamma}$')

    #plt.plot(df_reactions['x_ccw'], df_reactions['pdf_ccw'])
    df_reactions = df_reactions[df_reactions['x_log_ccw'] >= 1]
    x_diff = np.diff(df_reactions['x_log_ccw'])
    x_diff = np.r_[x_diff, x_diff[-1]]
    df_reactions['pdf_log_ccw'] = df_reactions['cnt_log_ccw'] / x_diff / df_reactions['cnt_log_ccw'].sum()
    norm = (df_reactions['pdf_log_ccw'] * df_reactions['x_log_ccw']).sum()
    df_reactions['pdf_log_ccw'] /= norm
    #plt.plot(df_reactions['x_log_ccw'], df_reactions['pdf_log_ccw'], label='Алгоритм Гиллеспи', zorder=10)
    
    norm = (df_reactions_data['t'] * df_reactions_data['p']).sum()
    #df_reactions_data['p'] /= mnorm
    df_reactions_data = df_reactions_data[df_reactions_data['t'] >= 1]
    print(df_reactions_data['t'], df_reactions_data['p'])
    plt.plot(df_reactions_data['t'], df_reactions_data['p'], label='Алгоритм Гиллеспи', zorder=10)
    
    print()
    
    #df_markov['p'] /= mnorm
    plt.plot(df_markov['t'], df_markov['p'], label='Статистика полного подсчета', lw=4, zorder=5)
    
    plt.text(20, 1e-2, r'$\boldsymbol{\gamma}\mathbf{=2.18}$', color='#6A3D9A')
    #plt.plot([1, 1000], [1e-15, 1e5], color='white')
    plt.xscale('log')
    plt.yscale('log')
    plt.yticks([1e-15, 1e-10, 1e-5, 1, 1e5])
    plt.ylim([1e-15, 1e5])
    plt.xlim([0.5, 1e3])
    plt.xlabel('Длительность, $t$')
    plt.ylabel('$p(t)$', rotation=0)
    plt.legend()
    plt.grid('on')

    ax = plt.axes([0.28, 0.23, 0.25, 0.25])
    
    #ax.plot(df_binomial['Y'], df_binomial['p'], color='#1F78B4')
    ax.plot(df_y_init['Y'], df_y_init['p'], color='#1F78B4', lw=2)
    ax.set_ylim([-0.001, 0.13])
    ax.set_ylabel('$P_S(Y)$', rotation=0, loc='center')
    ax.yaxis.set_label_coords(-.45, .45)
    ax.set_xlabel('$Y$')
    ax.set_xticks(np.arange(0, 41, 10))
    ax.set_xlim([0, 45])
    

from matplotlib.backends.backend_pdf import PdfPages
path_figure = figures_dir / "gillespie_vs_full-counting"
print(path_figure)
with PdfPages(path_figure.with_suffix('.pdf')) as pdf:
    fig = make_plot()
    #fig.savefig(path_figure.with_suffix('.png'), bbox_inches='tight')
    #fig.savefig(path_figure.with_suffix('.svg'))
    pdf.savefig(fig, bbox_inches='tight')

gillespie_vs_full-counting
150      1.0000
151      1.0471
152      1.0965
153      1.1482
154      1.2023
         ...   
295    794.3300
296    831.7600
297    870.9600
298    912.0100
299    954.9900
Name: t, Length: 150, dtype: float64 150    0.13425
151    0.12567
152    0.11770
153    0.11015
154    0.10284
        ...   
295    0.00000
296    0.00000
297    0.00000
298    0.00000
299    0.00000
Name: p, Length: 150, dtype: float64



In [146]:
df_binomial, df_y_init, df_reactions, df_markov = read_data()

In [151]:
x_diff = np.diff(df_reactions['x_log_ccw'])
x_diff = np.r_[x_diff, x_diff[-1]]
    
df_reactions['pdf_log_ccw'] = df_reactions['cnt_log_ccw'] / x_diff / df_reactions['cnt_log_ccw'].sum()
norm = (df_reactions['pdf_log_ccw'] * df_reactions['x_log_ccw']).sum()
df_reactions['pdf_log_ccw'] /= norm
df_reactions['pdf_log_ccw']

0      6.902010e-02
1      6.763535e-02
2      6.645404e-02
3      6.507289e-02
4      6.331000e-02
           ...     
295    2.651082e-07
296    2.113495e-07
297    2.065386e-07
298    1.562611e-07
299    1.475799e-07
Name: pdf_log_ccw, Length: 300, dtype: float64

In [148]:
x_diff

array([0.0023293 , 0.00238356, 0.00243908, 0.00249589, 0.00255403,
       0.00261352, 0.00267439, 0.00273669, 0.00280043, 0.00286566,
       0.00293241, 0.00300072, 0.00307061, 0.00314214, 0.00321533,
       0.00329022, 0.00336686, 0.00344529, 0.00352554, 0.00360766,
       0.00369169, 0.00377768, 0.00386567, 0.00395572, 0.00404786,
       0.00414214, 0.00423863, 0.00433736, 0.00443839, 0.00454177,
       0.00464756, 0.00475582, 0.0048666 , 0.00497995, 0.00509595,
       0.00521465, 0.00533612, 0.00546041, 0.0055876 , 0.00571775,
       0.00585094, 0.00598722, 0.00612668, 0.00626939, 0.00641542,
       0.00656486, 0.00671777, 0.00687425, 0.00703437, 0.00719822,
       0.00736589, 0.00753746, 0.00771303, 0.00789269, 0.00807654,
       0.00826467, 0.00845717, 0.00865417, 0.00885575, 0.00906203,
       0.00927311, 0.00948911, 0.00971014, 0.00993631, 0.01016776,
       0.0104046 , 0.01064695, 0.01089495, 0.01114873, 0.01140841,
       0.01167415, 0.01194608, 0.01222434, 0.01250908, 0.01280

In [13]:
plt.show()

C:\Users\mike_live\AppData\Local\Temp\ipykernel_3340\2122436968.py:1: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
